In [1]:
import sys
import pandas as pd
import numpy as np
import scanpy as sc
import anndata as an
import glob
from scipy.sparse import csr_matrix

sc.settings.verbosity = 3  

In [2]:
dpath = "/scratch/indikar_root/indikar1/cstansbu/hematokytos/processed_anndata/"
file_list = glob.glob(f"{dpath}/*")
file_list

['/scratch/indikar_root/indikar1/cstansbu/hematokytos/processed_anndata/pellin.h5ad',
 '/scratch/indikar_root/indikar1/cstansbu/hematokytos/processed_anndata/weng_young1_all_t1.h5ad',
 '/scratch/indikar_root/indikar1/cstansbu/hematokytos/processed_anndata/iHSC.h5ad',
 '/scratch/indikar_root/indikar1/cstansbu/hematokytos/processed_anndata/sc_fib.h5ad',
 '/scratch/indikar_root/indikar1/cstansbu/hematokytos/processed_anndata/weng_young1_all_t2.h5ad',
 '/scratch/indikar_root/indikar1/cstansbu/hematokytos/processed_anndata/weng_old2_BMMC_HSPC.h5ad',
 '/scratch/indikar_root/indikar1/cstansbu/hematokytos/processed_anndata/weng_old1_BMMC_HSPC.h5ad',
 '/scratch/indikar_root/indikar1/cstansbu/hematokytos/processed_anndata/weng_young2_HSC.h5ad',
 '/scratch/indikar_root/indikar1/cstansbu/hematokytos/processed_anndata/weng_young2_all.h5ad',
 '/scratch/indikar_root/indikar1/cstansbu/hematokytos/processed_anndata/tabula_sapiens.h5ad']

In [3]:
adata = sc.read_h5ad(file_list[7])
sc.logging.print_memory_usage()
print(adata)
print()
adata.to_df().head()

Memory usage: current 0.57 GB, difference +0.57 GB
AnnData object with n_obs × n_vars = 3703 × 16916
    obs: 'n_genes'
    var: 'n_cells'
    uns: 'log1p'
    layers: 'counts'



gene_name,AL627309.1,AL627309.5,LINC01409,LINC01128,LINC00115,FAM41C,SAMD11,NOC2L,ISG15,C1orf159,...,MT-ND4,MT-ND5,MT-ND6,MT-CYB,MAFIP,AL354822.1,AL592183.1,AC240274.1,AC004556.3,AC007325.4
AAACAGCCAACCCTAA-3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,6.860966,6.860966,0.000000,7.266082,0.0,0.0,0.000000,0.0,0.00000,0.0
AAACAGCCATAGGCGA-3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,7.993460,7.182952,0.000000,7.875719,0.0,0.0,0.000000,0.0,0.00000,0.0
AAACATGCAGCTAACC-3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,7.172581,7.172581,5.231265,7.018559,0.0,0.0,4.543450,0.0,4.54345,0.0
AAACATGCAGGCTGTT-3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.99951,0.0,...,7.883495,7.391258,0.000000,7.478218,0.0,0.0,5.689281,0.0,4.99951,0.0
AAACCGCGTAAATTGC-3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,7.335024,7.180982,0.000000,7.468473,0.0,0.0,5.393020,0.0,0.00000,0.0


In [8]:
obs, var = sc.pp.calculate_qc_metrics(
    adata,
    layer='counts',
    inplace=False
)

obs = obs.reset_index(drop=False, names='obs_index')
var = var.reset_index(drop=False, names='var_index')

print(f"{obs.shape=}")
print(f"{var.shape=}")


obs.shape=(3703, 9)
var.shape=(16916, 7)


In [7]:
obs.head()

,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes
AAACAGCCAACCCTAA-3,2120,7.659643,4196.0,8.342125,20.567207,29.551954,40.395615,57.459485
AAACAGCCATAGGCGA-3,1568,7.358194,3040.0,8.019942,23.486842,35.098684,46.907895,64.868421
AAACATGCAGCTAACC-3,4150,8.331105,10751.0,9.282847,13.812669,21.244535,30.936657,46.730537
AAACATGCAGGCTGTT-3,2997,8.005701,6787.0,8.822912,18.992191,28.245175,38.293797,53.823486
AAACCGCGTAAATTGC-3,2077,7.639161,4569.0,8.427268,23.134165,34.646531,45.743051,61.348216


In [4]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [ ]:
break

In [ ]:
def skeletonize(adata):
    """
    Reduces the size of an AnnData object by removing non-essential data.

    Args:
        adata: An AnnData object.

    Returns:
        A streamlined AnnData object.
    """

    # Remove unneeded annotations
    for attr in ['uns', 'obsm', 'obsp', 'varm']:
        if hasattr(adata, attr):
            delattr(adata, attr)

    # Handle layers
    if 'counts' in adata.layers:
        adata.X = adata.layers['counts'].copy()
    elif 'raw_counts' in adata.layers:
        adata.X = adata.layers['raw_counts'].copy()
    del adata.layers

    # Make sparse
    adata.X = csr_matrix(adata.X)

    # Handle var
    if 'gene_name' in adata.var.columns:
        adata.var_names = adata.var['gene_name']
    del adata.var

    # Handle obs
    del adata.obs

    return adata
    
    
    
    
adata = skeletonize(adata)
sc.logging.print_memory_usage()
adata

In [ ]:
adata.to_df().head()

In [ ]:
int(eval('1e6'))

In [ ]:
break